Redes Neuronales para Lenguaje Natural, 2025

---
# Laboratorio 2

En este laboratorio construiremos un sistema de Question Answering (QA) utilizando el método de Retrieval-Augmented Generation (RAG), que implica el uso de un paso de recuperación de información y un paso de generación de respuesta con LLM.

**Entrega: 18/11**

**Se debe entregar un archivo zip que contenga:**
* Este notebook de Python (.ipynb) completo.
* Los documentos obtenidos y utilizados como fuentes de información según se explica en la parte 1 (opcionalmente se puede entregar un archivo CSV con los textos de cada documento).
* Archivo CSV con el conjunto de preguntas y respuestas como se explica en la parte 5.

**No olvidar mantener todas las salidas de cada región de código en el notebook!**

---



In [ ]:
#@title Instalar librerias
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install sentence-transformers
!pip install evaluate
!pip install bert_score
!pip install wikipedia-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=acdb4a18ce5aa97346c71f43f3335b9825f2c9cdc40ce570a1c73a7ff0ddee10
  Stored in directory: /root/.cache/pip/wheels/33/3c/79/b36253689d838af4a0539782853ac3cc38a83a6591ad570dde
Successfully built wikipedia-api


In [ ]:
#@title Estilo de salida de colab
from IPython.display import HTML, display
pre_run_cell_fn = lambda: display(HTML('''<style> pre {white-space: pre-wrap;}</style>'''))
get_ipython().events.register('pre_run_cell', pre_run_cell_fn)

## Hyperparámetros

In [ ]:

# =========================================================================== #
# Hyperparámetros que se van a usar en las siguientes partes, en particular
# para experimentación en la parte 5.
#
# Los definimos aquí para poder modificarlos fácilmente en los experimentos
EMBEDDING="intfloat/multilingual-e5-large"
K_CHUNKS = 10
CHUNK_SIZE = 500
# Balancear entre preguntas más verborrágicas (puede alucinar)
# o más escuetas (pérdida de información)
MAX_TOKENS = 150
# SYSTEM_PROMPT = """
# Eres un asistente experto en responder preguntas.
# Tu tarea es responder la pregunta del usuario basándote única y exclusivamente en el contexto proporcionado.
# Si la información no se encuentra en el contexto, debes responder:
# "Lo siento, no cuento con información para responder esa pregunta."
# No inventes información. Sé conciso y directo.
# """
# Prompt ajustado al dominio:
SYSTEM_PROMPT = """
Eres un asistente experto en responder preguntas sobre perros, utilizando exclusivamente la información proveniente de artículos de Wikipedia.

INSTRUCCIONES:
1. Usa únicamente la información disponible en el contexto proporcionado (fragmentos del corpus de Wikipedia sobre perros).
2. No agregues, inventes ni completes datos que no estén en el contexto.
3. Si el contexto no contiene información suficiente para responder, di exactamente:
   "Lo siento, no cuento con información para responder esa pregunta."
4. Responde con un estilo claro, directo y enciclopédico, similar al de Wikipedia.
5. Evita opiniones o especulaciones.
6. Si hay varios fragmentos relevantes, combina la información de manera coherente y resumida.
7. Mantén el idioma de la pregunta original del usuario (español o inglés).
"""

# Other hyperparameters
# chunk_overlap=50,       # Solapamiento de chunks
# separators=["\n\n", "\n", ". ", ", ", " "]

# =========================================================================== #


## Parte 1: Procesamiento de los documentos

En esta parte, cada grupo deberá construir y procesar su conjunto de documentos. Esto consiste de los siguientes pasos:

* Elegir un tema dentro de un dominio específico sobre el que trabajar.
* Obtener al menos 5 documentos en español que contengan información sobre el tema elegido.
* Procesar cada documento para extraer el texto del formato original a un string en Python (por ejemplo, extraer el texto de un PDF).

El resultado de esta parte debe ser una lista cargada en memoria que contenga el texto (string) de cada uno de los documentos elegidos.

**Sugerencias:**
* Se recomienda utilizar artículos de wikipedia para simplificar la etapa de extracción del texto (ver la librería [wikipedia-api](https://github.com/martin-majlis/Wikipedia-API/)).
* Opcionalmente puede utilizar documentos PDF, páginas web u otros formatos. En estos casos se sugiere:
  * Utilizar la librería PyPDF2 para procesar documentos PDF.
  * Utilizar la librería LangChain para procesar páginas web, en particular la clase Html2TextTransformer, que convierte HTML a Markdown ([ejemplo de uso](https://python.langchain.com/v0.2/docs/integrations/document_transformers/html2text/)).
* Puede ser conveniente guardar el resultado del procesamiento de los documentos en un archivo CSV (donde cada fila corresponde al texto de un documento) para no tener que repetir este proceso cada vez que se ejecuta el notebook, y en su lugar cargar el archivo CSV.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import wikipediaapi
import pandas as pd
import os

WIKI_PAGES = [
    'Raza de perro',
    'Golden retriever',
    'Husky siberiano',
    'Caniche',
    'Perro cimarrón uruguayo'
]
CSV_FILENAME = 'documentos_wikipedia.csv'

if os.path.exists(CSV_FILENAME):
  # Se carga el CSV local
  df = pd.read_csv(CSV_FILENAME)
  loaded_documents = df['text'].fillna('').tolist()
else:
  # Se descargan los documentos de wikipedia
  wiki_wiki = wikipediaapi.Wikipedia(
      'RNLN_Lab2_2025_Student (urielmelodi@outlook.es)', 'es'
  )

  doc_data = []
  print("Documentos:\n")
  for i, title in enumerate(WIKI_PAGES):
    page = wiki_wiki.page(title)
    if page.exists():
      print(f"  Documento {i+1} - '{title}' ({len(page.text)} caracteres).")
      doc_data.append({'title': title, 'text': page.text})
    else:
      print(f"  No existe la página '{title}'.")

  df = pd.DataFrame(doc_data)
  df.to_csv(CSV_FILENAME, index=False)
  loaded_documents = df['text'].tolist()

Documentos:

  Documento 1 - 'Raza de perro' (8145 caracteres).
  Documento 2 - 'Golden retriever' (15030 caracteres).
  Documento 3 - 'Husky siberiano' (14242 caracteres).
  Documento 4 - 'Caniche' (13624 caracteres).
  Documento 5 - 'Perro cimarrón uruguayo' (8236 caracteres).


Los textos resultantes deben estar almacenados en la variable `documents`:

In [ ]:
documents = loaded_documents
print(f"\nTotal de documentos: {len(documents)}")


Total de documentos: 5


## Parte 2: Chunking

Una vez que se obtiene el texto de cada documento, se debe realizar la etapa de _chunking_. Esta etapa consiste en dividir cada texto en segmentos más chicos a los que llamamos _chunks_.

Realizar la etapa de _chunking_ de forma automática utilizando un método simple que permita obtener _chunks_ de un largo aproximado de 500 caracteres.

Puede probar con dividir a nivel de caracteres, palabras o incluso párrafos, teniendo en cuenta que el largo de cada _chunk_ no debería exceder demasiado los 500 caracteres.

**Sugerencias:**
* Puede utilizar los splitters disponibles en LangChain ([documentación](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/)) como RecursiveCharacterTextSplitter, aunque no es obligatorio y también es correcto hacer una implementación propia.
* Tener en cuenta que esta etapa es crucial en el resultado final. Cuanto más contextualizados queden los *chunks*, mejor será el rendimiento de la etapa de recuperación de información. Es conveniente minimizar la división de palabras (o párrafos) por la mitad.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Se usan separadores que tienen sentido para texto de Wikipedia
# Los separados son jerárquicos: parrafos -> lineas -> puntos -> comas -> espacios
def chunk_text(text, chunk_size_param, chunk_overlap_param, separators_param):
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=chunk_size_param,
      chunk_overlap=chunk_overlap_param,       # Solapamiento de chunks
      length_function=len,
      separators=separators_param
  )
  chunks = text_splitter.split_text(text)
  return chunks

In [ ]:
chunks = []
# Default values for chunking, can be changed in experiments
default_chunk_size = CHUNK_SIZE
default_chunk_overlap = 50
default_separators = ["\n\n", "\n", ". ", ", ", " "]

for i, document in enumerate(documents):
  print(f"Documento {i+1}/{len(documents)}:")
  doc_chunks = chunk_text(document, default_chunk_size, default_chunk_overlap, default_separators)
  chunks.extend(doc_chunks)
  print(f"  {len(doc_chunks)} chunks.")

print(f"\nTotal de chunks: {len(chunks)}")
if chunks:
    print(f"\nEjemplo de Chunk (Chunk 0)")
    print(chunks[0])
    print(f"\nLargo del chunk 0: {len(chunks[0])} caracteres.")
    print(f"\nLargo máximo de chunk: {max([len(c) for c in chunks])} caracteres.")

Documento 1/5:
  24 chunks.
Documento 2/5:
  53 chunks.
Documento 3/5:
  47 chunks.
Documento 4/5:
  37 chunks.
Documento 5/5:
  24 chunks.

Total de chunks: 185

Ejemplo de Chunk (Chunk 0)
Una raza de perro o raza canina es un grupo de perros que tienen características muy similares o casi idénticas en su aspecto o comportamiento o generalmente en ambos, sobre todo porque vienen de un sistema selecto de antepasados que tenían las mismas características. Los perros han sido apareados selectivamente para conseguir características específicas durante miles de años.

Largo del chunk 0: 379 caracteres.

Largo máximo de chunk: 497 caracteres.


## Parte 3: Recuperación de información

En esta parte vamos a implementar el método de recuperación de información que nos permitirá obtener los _chunks_ más relevantes para la pregunta.

En primer lugar, cargamos el modelo Bi-Encoder que utilizaremos para generar los embeddings utilizando la librería sentence_transformers.

Se utiliza el modelo multilingüe [intfloat/multilingual-e5-large](https://huggingface.co/intfloat/multilingual-e5-large), fine-tuning del modelo `xlm-roberta-large` para la tarea de generación de sentence embeddings.

Se pueden explorar otros modelos Bi-Encoder, e incluso modelos Cross-Encoder o del tipo ColBERT. En HuggingFace se puede consultar el siguiente [leaderboard](https://huggingface.co/spaces/mteb/leaderboard) que compara varios modelos de este tipo en diferentes tareas.

In [ ]:
from sentence_transformers import SentenceTransformer

model_emb = SentenceTransformer(EMBEDDING)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

A continuación se debe generar las representaciones vectoriales para todos los _chunks_ ([ejemplo de uso](https://huggingface.co/intfloat/multilingual-e5-large#support-for-sentence-transformers)).

**Observación:** El modelo que estamos usando espera que los _chunks_ comiencen con el prefijo `passage: ` por lo que será necesario agregarlo al inicio de todos los _chunks_.

In [ ]:
import numpy as np
import os
import numpy as np

#Agregar el prefijo "passage"
new_chunks = [f"passage: {c}" for c in chunks]

#Generar embeddings
chunk_embeddings = model_emb.encode(
    new_chunks,
    convert_to_tensor=True
)

#Convierto embeddings
chunk_embeddings_np = chunk_embeddings.cpu().numpy()

Por último, se debe implementar el algoritmo de búsqueda de los embeddings más cercanos para un embedding dado.

**Sugerencias:**
* Utilizar la clase NearestNeighbors de sklearn ([documentación](https://scikit-learn.org/dev/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)).

In [ ]:
from sklearn.neighbors import NearestNeighbors
#NearestNeighbors (metodo para calcular cercania)
nn_search = NearestNeighbors(n_neighbors=5, metric='cosine', algorithm='brute')

#Entrenamiento
nn_search.fit(chunk_embeddings_np)

#Funcion de busqueda (k mas cercanos al embedding)
def retrieve_chunks(embedding, k=3):
  #El embedding debe ser un array 2D (precondicion de .kneighbors)
  if embedding.ndim == 1:
    embedding = embedding.reshape(1, -1)


  #Busqueda (indices son los indices en el array chunks)
  distances, indices = nn_search.kneighbors(embedding, n_neighbors=k)

  #Obtengo textos usando los indices
  retrieved_chunk_texts = [chunks[i] for i in indices[0]]

  return retrieved_chunk_texts, indices[0], distances[0]

## Parte 4: Generación de respuestas

### Configuración de LLM

Utilizaremos el modelo **Llama 3.1** de Meta a través de la plataforma [HuggingFace](https://huggingface.co/). Para poder usar este modelo en HuggingFace es necesario seguir los siguientes pasos:

- Crearse una cuenta de HuggingFace (https://huggingface.co/)
- Aceptar los términos para usar el modelo en HuggingFace, que aparecen en el siguiente enlace: https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct
- Crear un token de HuggingFace con permiso de lectura siguiendo el siguiente enlace: https://huggingface.co/settings/tokens
- Ejecutar la siguiente celda e ingresar el token creado.

In [ ]:
# Conectarse a HuggingFace
from huggingface_hub import notebook_login

notebook_login()


def create_prompt(question):

  # Formato esperado por cada modelo
  format_by_embedding ={
      "intfloat/multilingual-e5-large" : f"query: {question}"
  }

  # Preparar la pregunta para el embedding
  query_prefixed = format_by_embedding[EMBEDDING]

  # Generar embedding para la pregunta
  question_embedding = model_emb.encode(query_prefixed)

  # Convertir a numpy 2D para sklearn (NearestNeighbors)
  if not isinstance(question_embedding, np.ndarray):
      question_embedding = question_embedding.cpu().numpy()

  if question_embedding.ndim == 1:
      question_embedding = question_embedding.reshape(1, -1)

  # Recuperar chunks relevantes usando la función de la parte 3
  try:
    retrieved_chunks, _, _ = retrieve_chunks(question_embedding, k=K_CHUNKS)
  except Exception as e:
    print(f"Error al recuperar chunks (¿'nn_search' o 'chunks' no definidos?): {e}")
    # Fallback si falla la recuperación (depuración)
    retrieved_chunks = ["Error: No se pudo recuperar el contexto."]

  # Construir el contexto (separando los chunks)
  context = "\n\n---\n\n".join(retrieved_chunks)

  # Crear el prompt para Llama 3.1 (Formato Chat)
  # Formateamos el contexto y la pregunta para el usuario
  user_message = f"""
**Contexto:**
{context}

**Pregunta:**
{question}
"""

  # Usamos el template de chat de Llama 3.1
  # La función 'get_response' usa un pipeline de text-generation,
  # que maneja correctamente esta lista de mensajes.
  messages = [
      {"role": "system", "content": SYSTEM_PROMPT},
      {"role": "user", "content": user_message}
      # El pipeline añadirá automáticamente el {"role": "assistant"}
  ]

  return messages

A continuación se inicializan el tokenizer y el modelo cuantizado a 4 bits.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

def load_model(model_path):
  # Inicializar el tokenizer y el modelo
  return AutoTokenizer.from_pretrained(
    model_path
  ), AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
  )

#En lugar de descargar el modelo completo de llama desde el repositorio oficial,
#optamos por descargar una version que ya este cuantizada (es lo mismo, pero la descarga es mas rapida)
tokenizer, model = load_model("unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Creamos ahora dos funciones auxiliares que usaremos para la generación de las respuestas.

In [ ]:
# Generar respuesta
from transformers import GenerationConfig, pipeline

def get_response(prompt, temp=0.0) :
  # Configuración de temperatura
  generation_config = GenerationConfig(
    temperature = temp if temp > 0 else None,
    do_sample = temp > 0
  )

  # Inicializar pipeline para generación de texto
  pipe = pipeline(
    "text-generation",
    model=model,
    config=generation_config,
    tokenizer=tokenizer,
    pad_token_id = tokenizer.eos_token_id
  )

  # Generar texto
  output = pipe(
      prompt,
      return_full_text=False,
      max_new_tokens=MAX_TOKENS
    )

  return output[0]['generated_text']

### Crear prompt y generar respuesta

Escribir la función `create_prompt(question)` que dada una pregunta, genere la prompt que se utilizará para generar la respuesta. Tener en cuenta que se debe realizar la búsqueda semántica de los _chunks_ más cercanos a la pregunta utilizando lo implementado en la parte 3.

**Observación:** Al igual que para los _chunks_, el modelo Bi-Encoder espera que la pregunta comience con un prefijo especial: `query: ` por lo que será necesario agregarlo al inicio de la pregunta para generar el embedding.

**Sugerencias:**
* Puede probar con distintas cantidades de _chunks_ recuperados, pero se sugiere comenzar con 3. Tener en cuenta que más _chunks_ recuperados y agregados en la prompt implica mayor uso de memoria en inferencia.
* Utilizar la función `apply_chat_template` del tokenizer para aplicar el template correcto del modelo Llama 3.1.

In [ ]:
question = "¿Qué es el Test de Turing y quién lo propuso?"

# 1. Crear el prompt (que ahora es una lista de mensajes)
prompt = create_prompt(question) # 'prompt' es la variable 'messages'

# 2. Imprimir el prompt
print("--- PROMPT (formato lista de mensajes) ---")
import json
print(json.dumps(prompt, indent=2, ensure_ascii=False))

# (Opcional) Imprimir cómo se verá el string que procesará el modelo
print("\n--- PROMPT (string renderizado por el tokenizer) ---")
prompt_string_view = tokenizer.apply_chat_template(
    prompt,
    tokenize=False,
    add_generation_prompt=True # Simula el inicio de la respuesta del asistente
)
print(prompt_string_view)


# 3. Generar la respuesta
print("\n--- RESPUESTA GENERADA ---")
# 'get_response' acepta la lista 'prompt' (messages)
response = get_response(prompt)
print(response)

# return prompt

Device set to use cuda:0


--- PROMPT (formato lista de mensajes) ---
[
  {
    "role": "system",
    "content": "\nEres un asistente experto en responder preguntas sobre perros, utilizando exclusivamente la información proveniente de artículos de Wikipedia.\n\nINSTRUCCIONES:\n1. Usa únicamente la información disponible en el contexto proporcionado (fragmentos del corpus de Wikipedia sobre perros).\n2. No agregues, inventes ni completes datos que no estén en el contexto.\n3. Si el contexto no contiene información suficiente para responder, di exactamente:\n   \"Lo siento, no cuento con información para responder esa pregunta.\"\n4. Responde con un estilo claro, directo y enciclopédico, similar al de Wikipedia.\n5. Evita opiniones o especulaciones.\n6. Si hay varios fragmentos relevantes, combina la información de manera coherente y resumida.\n7. Mantén el idioma de la pregunta original del usuario (español o inglés).\n"
  },
  {
    "role": "user",
    "content": "\n**Contexto:**\n. Es por ello que se comenzó a 

Probar la prompt anterior con un ejemplo.

In [ ]:
question = "¿Quién lideró al Batallón de Blandengues?
prompt = create_prompt(question)
print("PROMPT:")
print(prompt)

print("\nRESPUESTA:")
print(get_response(prompt))

Device set to use cuda:0


Object `Blandengues` not found.
PROMPT:
[{'role': 'system', 'content': '\nEres un asistente experto en responder preguntas sobre perros, utilizando exclusivamente la información proveniente de artículos de Wikipedia.\n\nINSTRUCCIONES:\n1. Usa únicamente la información disponible en el contexto proporcionado (fragmentos del corpus de Wikipedia sobre perros).\n2. No agregues, inventes ni completes datos que no estén en el contexto.\n3. Si el contexto no contiene información suficiente para responder, di exactamente:\n   "Lo siento, no cuento con información para responder esa pregunta."\n4. Responde con un estilo claro, directo y enciclopédico, similar al de Wikipedia.\n5. Evita opiniones o especulaciones.\n6. Si hay varios fragmentos relevantes, combina la información de manera coherente y resumida.\n7. Mantén el idioma de la pregunta original del usuario (español o inglés).\n'}, {'role': 'user', 'content': '\n**Contexto:**\n. Es por ello que se comenzó a trabajar en la cría de un perro

## Parte 5: Evaluación
A continuación vamos a evaluar la solución construida. Para ello, se deben seguir los siguientes pasos:

* Construir un conjunto de evaluación de forma manual que contenga al menos 12 preguntas y respuestas con las siguientes características:
  * Al menos 3 preguntas deben necesitar información presente en más de un _chunk_ para ser respondidas correctamente.
  * Al menos 3 preguntas no deben estar relacionadas con el dominio, y su respuesta de referencia debe ser algo similar a: "Lo siento, no cuento con información para responder esa pregunta."
* El conjunto debe estar en un archivo CSV llamado testset.csv, con las columnas "question" y "answer".

Se deberá realizar al menos tres experimentos diferentes y evaluar sobre el mismo conjunto de test con la métrica BERTScore. Los experimentos deben variar en al menos uno de los siguientes elementos:
* Método de chunking
* Modelo (o método) de retrieval
* Modelo de generación (LLM)
* Método de prompting (se puede probar con few-shot, chain of thought, etc)
* Otros aspectos que considere relevantes a probar

A continuación se definen funciones auxiliares para la evaluación.


In [ ]:
!pip install evaluate
import evaluate
import numpy as np
from tqdm.notebook import tqdm
from sklearn.neighbors import NearestNeighbors # Import NearestNeighbors here to ensure it's available

def generate_predictions(questions, k_chunks, system_prompt, chunk_size, chunk_overlap, separators, doc_texts):
  # Re-chunk the documents with the specified parameters for this experiment
  current_chunks = []
  for doc in doc_texts:
      current_chunks.extend(chunk_text(doc, chunk_size, chunk_overlap, separators))

  # Re-generate chunk embeddings for the new chunks
  new_chunks_prefixed = [f"passage: {c}" for c in current_chunks]
  current_chunk_embeddings = model_emb.encode(
      new_chunks_prefixed,
      convert_to_tensor=True
  )
  current_chunk_embeddings_np = current_chunk_embeddings.cpu().numpy()

  # Re-fit the NearestNeighbors model with the new embeddings
  current_nn_search = NearestNeighbors(n_neighbors=5, metric='cosine', algorithm='brute')
  current_nn_search.fit(current_chunk_embeddings_np)

  def retrieve_chunks_for_exp(embedding, k_val):
      if embedding.ndim == 1:
          embedding = embedding.reshape(1, -1)
      distances, indices = current_nn_search.kneighbors(embedding, n_neighbors=k_val)
      retrieved_chunk_texts = [current_chunks[i] for i in indices[0]]
      return retrieved_chunk_texts, indices[0], distances[0]

  # Adjust create_prompt to use the experimental retrieve_chunks and system_prompt
  def create_prompt_for_exp(question_text):
      format_by_embedding ={"intfloat/multilingual-e5-large" : f"query: {question_text}"}
      query_prefixed = format_by_embedding[EMBEDDING]
      question_embedding = model_emb.encode(query_prefixed)
      if not isinstance(question_embedding, np.ndarray):
          question_embedding = question_embedding.cpu().numpy()
      if question_embedding.ndim == 1:
          question_embedding = question_embedding.reshape(1, -1)

      retrieved_chunks_exp, _, _ = retrieve_chunks_for_exp(question_embedding, k_val=k_chunks)
      context = "\n\n---\n\n".join(retrieved_chunks_exp)
      user_message = f"""
**Contexto:**
{context}

**Pregunta:**
{question_text}
"""
      messages = [
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_message}
      ]
      return messages

  prompts = [create_prompt_for_exp(question) for question in questions]
  predictions = [get_response(prompt) for prompt in tqdm(prompts)]
  return predictions

def evaluate_predictions(predictions, references):
  bertscore = evaluate.load("bertscore")
  results = bertscore.compute(predictions=predictions, references=references, lang='es')

  print(f"BERTScore P: {np.array(results['precision']).mean():.3f}")
  print(f"BERTScore R: {np.array(results['recall']).mean():.3f}")
  print(f"BERTScore F1: {np.array(results['f1']).mean():.3f}")

In [ ]:
import pandas as pd

#Definomos datos sobre razas de perro
test_data = {
    "question": [
        #Simples (requieren un unico chunk)
        "¿Cuál es el temperamento típico del Golden Retriever?",
        "¿Para qué se crió originalmente el Husky Siberiano?",
        "¿Qué tipos de Caniche existen según su tamaño?",
        "¿De dónde es originario el Perro Cimarrón Uruguayo?",
        "¿Qué es una raza de perro?",
        "¿El Husky Siberiano necesita mucho ejercicio?",
        #Complejas (requieren multiples chunks)
        "Compara el temperamento y las necesidades de ejercicio del Golden Retriever y el Husky Siberiano.",
        "¿En qué se diferencian el Caniche y el Perro Cimarrón Uruguayo en cuanto a su origen y función?",
        "¿Cómo se aplican los estándares de una raza de perro, según la definición general, al caso específico del Caniche?",
        #Fuera de tema (no hay informacion al respecto)
        "¿Cuál es la capital de Francia?",
        "¿Quién escribió Don Quijote de la Mancha?",
        "¿Cuál es la receta para hacer una torta de chocolate?"
    ],
    #Respuestas de referencia
    "answer": [
        #Simples
        "El Golden Retriever es conocido por su temperamento amigable, inteligente, leal y paciente. Es un perro confiable y bueno con los niños y otras mascotas.",
        "El Husky Siberiano fue criado originalmente por el pueblo Chukchi en Siberia para tirar de trineos en condiciones de frío extremo y largas distancias.",
        "Existen cuatro variedades de tamaño del Caniche (o Poodle): gigante (estándar), mediano, enano y toy.",
        "El Perro Cimarrón Uruguayo es una raza de perro originaria de Uruguay, descendiente de los perros traídos por los colonizadores españoles.",
        "Una raza de perro es un grupo de perros que tienen características fenotípicas muy similares y homogéneas, como tamaño, forma y pelaje, que se transmiten a sus descendientes.",
        "Sí, el Husky Siberiano es una raza de alta energía que requiere una cantidad significativa de ejercicio diario, como correr o tirar de trineos, para mantenerse saludable y evitar problemas de comportamiento.",
        #Complejas
        "El Golden Retriever es amigable, leal y tiene necesidades de ejercicio moderadas (como juegos de buscar), mientras que el Husky Siberiano es más independiente, enérgico y requiere un alto nivel de ejercicio intenso debido a su origen como perro de trineo.",
        "El Caniche (Poodle) tiene un origen disputado entre Francia y Alemania, criado como cobrador de agua, mientras que el Perro Cimarrón Uruguayo es originario de Uruguay, desarrollado como perro de trabajo, guardia y caza mayor en el campo.",
        "Una raza de perro se define por características homogéneas. En el Caniche, esto incluye su pelaje rizado y denso, su inteligencia y sus diferentes tamaños (toy, enano, mediano, gigante), todos fijados en un estándar de raza.",
        #Fuera de tema
        "Lo siento, no cuento con información para responder esa pregunta.",
        "Lo siento, no cuento con información para responder esa pregunta.",
        "Lo siento, no cuento con información para responder esa pregunta."
    ]
}

#Guardar en CSV
df_testset = pd.DataFrame(test_data)
df_testset.to_csv("testset.csv", index=False)

print("Archivo 'testset.csv' creado.")

#Leer el conjunto de test
df = pd.read_csv("testset.csv")

#Separar en preguntas y respuestas
questions = df["question"].tolist()
references = df["answer"].tolist()

print(f"Cargadas {len(questions)} preguntas y {len(references)} respuestas de referencia.")

Archivo 'testset.csv' creado.
Cargadas 12 preguntas y 12 respuestas de referencia.


Evalúe los experimentos realizados.

In [ ]:
# Definimos los prompts del sistema para los experimentos
system_prompt_strict = """
Eres un asistente experto en responder preguntas sobre perros, utilizando exclusivamente la información proveniente de artículos de Wikipedia.

INSTRUCCIONES:
1. Usa únicamente la información disponible en el contexto proporcionado (fragmentos del corpus de Wikipedia sobre perros).
2. No agregues, inventes ni completes datos que no estén en el contexto.
3. Si el contexto no contiene información suficiente para responder, di exactamente:
   "Lo siento, no cuento con información para responder esa pregunta."
4. Responde con un estilo claro, directo y enciclopédico, similar al de Wikipedia.
5. Evita opiniones o especulaciones.
6. Si hay varios fragmentos relevantes, combina la información de manera coherente y resumida.
7. Mantén el idioma de la pregunta original del usuario (español o inglés).
"""

system_prompt_basic = """
Eres un asistente de IA. Responde la pregunta del usuario usando el contexto proporcionado.
"""

# Definir parámetros de chunking para experimentos
default_chunk_size = 500
default_chunk_overlap = 50
default_separators = ["\n\n", "\n", ". ", ", ", " "]

#Listas para guardar los resultados numéricos
exp_results = []

#1-Base (k=3, Prompt Estricto)
print("\nIniciando Experimento 1: Base (k=3, Prompt Estricto, Chunking por defecto)")
#Generar predicciones
predictions_exp1 = generate_predictions(questions, k_chunks=3, system_prompt=system_prompt_strict,
                                        chunk_size=default_chunk_size, chunk_overlap=default_chunk_overlap, separators=default_separators, doc_texts=documents)
#Evaluar
print("\nResultados Exp 1:")
evaluate_predictions(predictions_exp1, references)
#Guardar resultados (para la tabla)
results_exp1 = evaluate.load("bertscore").compute(predictions=predictions_exp1, references=references, lang='es')
exp_results.append(results_exp1)


#2-Más Contexto (k=5, Prompt Estricto)
print("\nIniciando Experimento 2: Más Contexto (k=5, Prompt Estricto, Chunking por defecto)")
#Generar predicciones
predictions_exp2 = generate_predictions(questions, k_chunks=5, system_prompt=system_prompt_strict,
                                        chunk_size=default_chunk_size, chunk_overlap=default_chunk_overlap, separators=default_separators, doc_texts=documents)
#Evaluar
print("\nResultados Exp 2:")
evaluate_predictions(predictions_exp2, references)
#Guardar resultados
results_exp2 = evaluate.load("bertscore").compute(predictions=predictions_exp2, references=references, lang='es')
exp_results.append(results_exp2)


#3-Prompt Básico (k=3, Prompt Simple)
print("\nIniciando Experimento 3: Prompt Básico (k=3, Prompt Simple, Chunking por defecto)")
#Generar predicciones
predictions_exp3 = generate_predictions(questions, k_chunks=3, system_prompt=system_prompt_basic,
                                        chunk_size=default_chunk_size, chunk_overlap=default_chunk_overlap, separators=default_separators, doc_texts=documents)
#Evaluar
print("\nResultados Exp 3:")
evaluate_predictions(predictions_exp3, references)
#Guardar resultados
results_exp3 = evaluate.load("bertscore").compute(predictions=predictions_exp3, references=references, lang='es')
exp_results.append(results_exp3)

# 4 - Chunking diferente (k=3, Prompt Estricto, Chunk_size 250)
print("\nIniciando Experimento 4: Chunking Diferente (k=3, Prompt Estricto, Chunk_size=250)")
predictions_exp4 = generate_predictions(questions, k_chunks=3, system_prompt=system_prompt_strict,
                                        chunk_size=250, chunk_overlap=25, separators=default_separators, doc_texts=documents)
print("\nResultados Exp 4:")
evaluate_predictions(predictions_exp4, references)
results_exp4 = evaluate.load("bertscore").compute(predictions=predictions_exp4, references=references, lang='es')
exp_results.append(results_exp4)

#5-Separators diferente (k=3, Prompt Estricto)
print("\nIniciando Experimento 5: Separators diferente (k=3, Prompt Estricto, Chunking por defecto)")
#Generar predicciones
predictions_exp5 = generate_predictions(questions, k_chunks=3, system_prompt=system_prompt_strict,
                                        chunk_size=default_chunk_size, chunk_overlap=default_chunk_overlap, separators=["\n\n", "\n"], doc_texts=documents)
#Evaluar
print("\nResultados Exp 5:")
evaluate_predictions(predictions_exp5, references)
#Guardar resultados (para la tabla)
results_exp5 = evaluate.load("bertscore").compute(predictions=predictions_exp5, references=references, lang='es')
exp_results.append(results_exp5)

#Probamos con un modelo pequeño
#6 - Base-gemma (k=3, Modelo de 1B de parametros)
print("\nIniciando Experimento 6: Base-gemma (k=3, Chunking por defecto)")
#Cargamos el modelo de gemma
tokenizer, model = load_model("unsloth/gemma-3-1b-it-bnb-4bit")
#Generar predicciones
predictions_exp6 = generate_predictions(questions, k_chunks=3, system_prompt=system_prompt_strict,
                                        chunk_size=default_chunk_size, chunk_overlap=default_chunk_overlap, separators=default_separators, doc_texts=documents)
#Evaluar
print("\nResultados Exp 6:")
evaluate_predictions(predictions_exp6, references)
#Guardar resultados (para la tabla)
results_exp6 = evaluate.load("bertscore").compute(predictions=predictions_exp6, references=references, lang='es')
exp_results.append(results_exp6)

#Imprimir resumen
print("\n\nResumen de Resultados para la Tabla")
print("| Exp | Descripción | P BERTScore | R BERTScore | F BERTScore |")
print("|-----|-------------|-------------|-------------|-------------|")
print("| 1 | Base (k=3, Prompt Estricto) | {:.3f} | {:.3f} | {:.3f} |".format(
    np.mean(exp_results[0]['precision']), np.mean(exp_results[0]['recall']), np.mean(exp_results[0]['f1'])
))
print("| 2 | Más Contexto (k=5, Prompt Estricto) | {:.3f} | {:.3f} | {:.3f} |".format(
    np.mean(exp_results[1]['precision']), np.mean(exp_results[1]['recall']), np.mean(exp_results[1]['f1'])
))
print("| 3 | Prompt Básico (k=3, Prompt Simple) | {:.3f} | {:.3f} | {:.3f} |".format(
    np.mean(exp_results[2]['precision']), np.mean(exp_results[2]['recall']), np.mean(exp_results[2]['f1'])
))
print("| 4 | Chunk Size 250 (k=3, Prompt Estricto) | {:.3f} | {:.3f} | {:.3f} |".format(
    np.mean(exp_results[3]['precision']), np.mean(exp_results[3]['recall']), np.mean(exp_results[3]['f1'])
))
print("| 5 | Separators diferente (k=3, Prompt Estricto) | {:.3f} | {:.3f} | {:.3f} |".format(
    np.mean(exp_results[4]['precision']), np.mean(exp_results[4]['recall']), np.mean(exp_results[4]['f1'])
))
print("| 6 | Base-gemma (k=3, Prompt Estricto) | {:.3f} | {:.3f} | {:.3f} |".format(
    np.mean(exp_results[5]['precision']), np.mean(exp_results[5]['recall']), np.mean(exp_results[5]['f1'])
))


Iniciando Experimento 1: Base (k=3, Prompt Estricto, Chunking por defecto)


  0%|          | 0/12 [00:00<?, ?it/s]

Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0



Resultados Exp 1:


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/965M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

  0%|          | 0/12 [00:00<?, ?it/s]

Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0



Resultados Exp 6:
BERTScore P: 0.750
BERTScore R: 0.769
BERTScore F1: 0.759


Resumen de Resultados para la Tabla
| Exp | Descripción | P BERTScore | R BERTScore | F BERTScore |
|-----|-------------|-------------|-------------|-------------|
| 1 | Base (k=3, Prompt Estricto) | 0.756 | 0.782 | 0.767 |
| 2 | Más Contexto (k=5, Prompt Estricto) | 0.791 | 0.827 | 0.809 |
| 3 | Prompt Básico (k=3, Prompt Simple) | 0.690 | 0.747 | 0.715 |
| 4 | Chunk Size 250 (k=3, Prompt Estricto) | 0.736 | 0.788 | 0.760 |
| 5 | Separators diferente (k=3, Prompt Estricto) | 0.764 | 0.785 | 0.772 |
| 6 | Base-gemma (k=3, Prompt Estricto) | 0.750 | 0.769 | 0.759 |


Reportar los resultados obtenidos en los experimentos realizados completando la siguiente tabla:

| Exp | Descripción | P BERTScore | R BERTScore | F BERTScore |
|-----|-------------|-------------|-------------|-------------|
| 1 | Base (k=3, Prompt Estricto) | 0.770 | 0.796 | 0.782 |
| 2 | Más Contexto (k=5, Prompt Estricto) | 0.763 | 0.830 | 0.794 |
| 3 | Prompt Básico (k=3, Prompt Simple) | 0.689 | 0.750 | 0.718 |
| 4 | Chunk Size 250 (k=3, Prompt Estricto) | 0.738 | 0.778 | 0.755 |
| 5 | Separators diferente (k=3, Prompt Estricto) | 0.756 | 0.785 | 0.768 |
| 6 | Base-gemma (k=3, Prompt Estricto) | 0.754 | 0.769 | 0.760 |

Responda las siguientes preguntas:

1. Explique brevemente las diferencias en los experimentos realizados, ¿Qué aspectos se varió en el pipeline de RAG?

2. ¿Son consistentes los resultados obtenidos con lo que esperaba?

3. ¿Le parece que la métrica BERTScore está capturando correctamente las diferencias de los distintos experimentos realizados?

# Explique brevemente las diferencias en los experimentos realizados, ¿Qué aspectos se varió en el pipeline de RAG?


## 1. Explicación de los experimentos realizados

Los aspectos del pipeline de RAG que se variaron en los experimentos fueron:
- Número de chunks recuperados (k)
  - Se comparó k = 3 vs k = 5.
  - Cambiar k afecta cuánta evidencia recibe el LLM: más contexto puede potencialmente proveer una mejor respuesta, pero también puede tener más ruido (lo que puede comprometer el rendimiento).
- Prompting (Prompt Estricto vs Prompt Simple)
  - El fin de es probar cuánto influye el control del modelo frente a la creatividad del modelo. Nuestra hipótesis, que queríamos comprobar, es que un promt menos escricto puede darle más libertad al modelo, y uno más escricto puede generar respuestas más precisas.
- Tamaño del chunk (500, 250)
  - El tamaño del chunk influye en la granularidad de la información y afecta driéctamente al contexto que llegará al LLM. Chunks más grandes entregan más información por fragmento, y chunks más pequeños pueden fragmentar la semántica y reducir la coherencia del contexto. Pero a la vez chunks demasiado grandes pueden incluir ruido e información irrelevante.
- Distintos separadores
  - Probamos con una lista mas restrictiva de los separadores, utilizando unicamente los saltos de linea y los parrafos.
- Cambio de modelo generativo (Llama vs Gemma)
  - Decidimos elegir otro modelo alternativo más pequeño al sugerido en la letra del laboratorio para comparar sus rendimientos. Comparamos lama/Meta-Llama-3.1-8B-Instruct contra unsloth/gemma-3-1b-it-bnb-4bit.


### Experimento 1
Configuración:
- k = 3 documentos recuperados.
- Prompt estricto: el modelo debe responder solo usando el contexto dado.
- Tamaño de chunk: valor “por defecto” del laboratorio (el que se definió inicialmente).
- Modelo generativo: Meta-Llama-3.1-8B-Instruct (o equivalente base del lab).

**Rol del experimento:** Este experimento es la línea base. Desde aquí se comparan el resto de las variantes: más contexto, prompt más laxo, chunk distinto, otro modelo.

### Experimento 2
Configuración:
- Todo igual que Experimento 1, excepto que se modifica la parte de recuperación (Retriever): se traen más chunks como contexto para la respuesta, k = 5 en lugar de 3.

**Objetivo:** Evaluar cómo impacta aumentar el número de documentos recuperados (más evidencia) manteniendo el mismo tipo de prompt y chunking.

### Experimento 3
Configuración:
- k = 3 (igual que en el Experimento 1).
- Prompt simple/básico: menos restrictivo, sin demasiados detalles.
- Chunk size: el mismo que en el Experimento 1.
- Modelo: el mismo que en el Experimento 1.

**Objetivo:** Medir cómo influye solo el diseño del prompt cuando el resto del pipeline (retriever, chunking, modelo) se mantiene igual.

### Experimento 4
Configuración:
- k = 3 (como en el Experimento 1).
- Prompt estricto (como en el Experimento 1).
- Tamaño de chunk = 250 tokens (más pequeño que el Experimento 1).
- Modelo: el mismo que en el Experimento 1.

**Objetivo:** Analizar el impacto del tamaño del chunk sobre: la calidad de los embeddings, lo que recupera el retriever, y el contexto final que llega al LLM.

### Experimento 5
Configuración:
- k = 3 (como en el Experimento 1).
- Prompt estricto (como en el Experimento 1).
- Chunk size: el mismo que en el Experimento 1.
- Modelo generativo: se cambia a Gemma, se mantiene el mismo retriever

**Objetivo:** Comparar el desempeño de otro modelo generativo en exactamente el mismo setup de RAG.

# 2. ¿Son consistentes los resultados obtenidos con lo que esperaba?

# Resultados esperados vs obtenidos

Los resultados del laboratorio siguen los patrones esperados.

### Aumentar k (3 → 5)

`El rendimiento aumento un 1.51%`

Aumento el F-BERTScore de 0.794 a 0.782, pero no es concluyente ya que al ejecutarlo multiples veces, no siempre mejora.

### Prompt Simple da peor rendimiento

`El rendimiento aumentó un 9.57%`

En este caso la diferencia en el rendimiento es más notorio, mustra un empeoramiento en  el F-score a 0.718. Tuvo peor rendimiento porque permitió que el modelo se apartara del contexto dado y generara respuestas más libres (mayor alucinación), más genéricas y menos alineadas con la referencia. También cabe aclarar que de todas formas las respuestas con un prompt simple fueron dentro de lo que se puede decir razonable.

### Chunk size más pequeño produce resultados ligeramente peores

`El rendimiento disminuyó un 3.45%`

Chunk 250 (F ≈ 0.755) es inferior al del Experimento 1. Esto nuevamente coincide con lo esperado: chunks más cortos fragmentan demasiado la información e implica menos cohesión semántica, y por lo tanto embeddings más débiles. Afectando negativamente la recuperación y la respuesta final.

### Separadores más estrictos

`El rendimiento disminuyó un 1.79%`

(F ≈ 0.768) es inferior al del Experimento 1. Nosotros esperabamos que al cortar los chunks en los cambios de linea y parrafo, iban a estar más contextualizados, lo cual mejoraría el rendimiento, pero no ocurrió.

### Cambiar a Gemma también reduce el score

`Se redujo el rendimiento en ~2.81%, F-BERTScore dió 0.760.`

Se podía esperar que diera un rendimiento menor al tratarse de un modelo más pequeño. En todo caso, el modelo dio muy buenos resultados considerando que solo tiene 1 B de parámetros, lo cual refleja que, para este tipo de tareas sencillas, no se requiere un gran poder de cómputo para obtener resultados satisfactorios.



# 3. ¿Le parece que la métrica BERTScore está capturando correctamente las diferencias de los distintos experimentos realizados?

# Evaluación de diferencias con BERTScore

Se puede decir que BERTScore en general marca las diferencias como vimos en los resultados anteriores, pero hay que comentar ciertos matices.

Si bien BERTScore es de las mejores metricas que hay para evaluar question answering, no debería usarse como métrica única para evaluar un sistema RAG, porque BERTScore no evalúa:
- Factualidad (puede coincidir semánticamente pero ser falso).
- Precisión de detalles concretos (p.ej., fechas, números, razas específicas).
- Estructura, exhaustividad y formato de la respuesta.
- Cumplimiento del prompt (por ejemplo, no alucinar).

Por eso un prompt simple tiene un F=0.716 que parece “aceptable”, cuando puede haber cometido errores importantes.

El BERTScore se puede complementar con
- Evaluación humana.
- Métricas de seguridad o alucinación como Hallucination Rate y Toxicity Score.
- LLM as a judge
